<a href="https://colab.research.google.com/github/amanteur/TDA_Cover_detection/blob/main/TDA_CoverDetection_ML_Models_Preprocessing_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.2.0

     |████████████████████████████████| 9.9MB 5.3MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.0 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [ ]:
!pip install gudhi

     |████████████████████████████████| 28.2MB 120kB/s 


In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import gudhi as gd
import gudhi.representations as gdr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Extracting data

In [ ]:
root_dir = '/content/drive/MyDrive/Colab Notebooks/CourseWork/pers_feat_datasets/'

In [ ]:
df_dict = {}
for _, _, files in os.walk(root_dir):
  for file in files:
    name = file.split('.')[0]
    df = pd.read_json(root_dir + file)
    df_dict [name] = df

#Test/train

In [ ]:
def split_data(df_dict, label_col='name', test_size=0.125, random_state=42):
  #extracting labels aka songs' names
  labels = set(df_dict[np.random.choice(list(df_dict))][label_col].tolist())
  #making train/test labels
  lbl_train, lbl_test = train_test_split(list(labels), test_size=test_size, random_state=random_state)

  train_test_dict = {}
  for name, df in df_dict.items():
    train_df = df[df[label_col].isin(lbl_train)]
    test_df = df[df[label_col].isin(lbl_test)]

    train_test_dict.setdefault(name,{})
    train_test_dict[name]['train'] = train_df
    train_test_dict[name]['test'] = test_df

  return train_test_dict

In [ ]:
train_test_dict = split_data(df_dict, test_size=0.15)
#example
train_test_dict['af_orig']['train'].shape, train_test_dict['af_orig']['test'].shape 

((137, 7), (27, 7))

#Making pairs

In [ ]:
def make_pairs(train_test_dict):
  pairs_dict = {}
  for name, train_test in train_test_dict.items():

    pairs_dict.setdefault(name,{})

    for tt, df in train_test.items():
      #resetting index in order to get id of song
      part_df = df.reset_index()
      pair = pd.merge(part_df, part_df, how='cross')
      #exclude rows of same songs
      pair = pair[pair.index_x != pair.index_y]

      #delete same rows
      pair['indices'] = list(zip(pair.index_x, pair.index_y))
      pair[['indices']] = pair[['indices']].applymap(frozenset)
      pair = pair.drop_duplicates(subset=['indices'])
      
      #adding predicting label
      pair['ref_cover'] = np.where((pair['name_x'] == pair['name_y']), 1, 0)

      #normalizing dataframe (because there are not so much "1" labels)
      pos_idx = pair.index[pair['ref_cover'] == 1]
      n_pos = pos_idx.shape[0]
      neg_idx = pair.index[pair['ref_cover'] == 0]
      np.random.seed(0)
      neg_idx_stay = np.random.choice(neg_idx, n_pos, replace=False)

      indices = np.concatenate([pos_idx, neg_idx_stay])

      output_pair = pair[pair.index.isin(indices)]

      #dropping unnecessary columns
      output_pair.drop(labels=['index_x', 'index_y', 'indices'], inplace=True, axis=1)

      #adding to dict
      pairs_dict[name][tt] = output_pair.reset_index(drop=True)

  return pairs_dict

In [ ]:
pairs_df_dict = make_pairs(train_test_dict)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.p

In [ ]:
pairs_df_dict['af_DTM']['train'].head()

,name_x,persistence_DTM_x,Landscape_x,Silhouette_x,Entropy_vector_x,Betti_curves_x,name_y,persistence_DTM_y,Landscape_y,Silhouette_y,Entropy_vector_y,Betti_curves_y,ref_cover
0,Blue_Collar_Man,"[[0.1774890428, 0.2075652335], [0.1992981601, ...","[[0.0028132621, 0.0056265242, 0.0084397862, 0....","[[0.0, 9.81306e-05, 0.00019626110000000002, 0....","[[0.3193268741, 0.3193268741, 0.3193268741, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Blue_Collar_Man,"[[0.14879702990000002, 0.174270759], [0.161238...","[[0.0026977515, 0.0053955031, 0.0080932546, 0....","[[0.0, 0.0001046298, 0.00020925970000000002, 0...","[[0.3615991845, 0.3615991845, 0.3615991845, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",1
1,Blue_Collar_Man,"[[0.14879702990000002, 0.174270759], [0.161238...","[[0.0026977515, 0.0053955031, 0.0080932546, 0....","[[0.0, 0.0001046298, 0.00020925970000000002, 0...","[[0.3615991845, 0.3615991845, 0.3615991845, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Downtown_Lights,"[[0.184942882, 0.2129234875], [0.1858301284000...","[[0.0018810603, 0.0037621207, 0.00564318100000...","[[0.0, 8.819060000000001e-05, 0.0001763811, 0....","[[0.1339225612, 0.1339225612, 0.1339225612, 0....","[[1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",0
2,Blue_Collar_Man,"[[0.14879702990000002, 0.174270759], [0.161238...","[[0.0026977515, 0.0053955031, 0.0080932546, 0....","[[0.0, 0.0001046298, 0.00020925970000000002, 0...","[[0.3615991845, 0.3615991845, 0.3615991845, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Thin_Line_Between_Love_And_Hate,"[[0.22275412360000002, 0.2645441979], [0.21683...","[[0.0019939, 0.0039877999, 0.0059816999, 0.007...","[[0.0, 9.51582e-05, 0.0001903165, 0.0002854747...","[[0.3535085821, 0.3535085821, 0.3535085821, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",0
3,Addicted_To_Love,"[[0.1101803537, 0.127565866], [0.1143794879000...","[[0.0015132986000000001, 0.0030265971000000003...","[[0.0, 6.98168e-05, 0.0001396336, 0.0002094505...","[[0.23833650520000002, 0.23833650520000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",Addicted_To_Love,"[[0.1568564542, 0.1837942471], [0.160458898700...","[[0.0028146417, 0.0056292833, 0.008443925, 0.0...","[[0.0, 0.0001569411, 0.00040158530000000003, 0...","[[0.3385326631, 0.7009968308, 0.7009968308, 0....","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0,...",1
4,Addicted_To_Love,"[[0.1568564542, 0.1837942471], [0.160458898700...","[[0.0028146417, 0.0056292833, 0.008443925, 0.0...","[[0.0, 0.0001569411, 0.00040158530000000003, 0...","[[0.3385326631, 0.7009968308, 0.7009968308, 0....","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0,...",Walking_After_Midnight,"[[0.14295036120000001, 0.1690991199], [0.13819...","[[0.0024385958000000003, 0.0048771916000000005...","[[0.0, 8.90445e-05, 0.000178089, 0.0002671335,...","[[0.3674337802, 0.3674337802, 0.3674337802, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0,...",0


#Mutual features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

def data_preprocessing(data):
  data_scaled = gdr.DiagramSelector(use=True, point_type="finite").fit_transform(data)
  data_scaled = gdr.DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())]).fit_transform(data_scaled)
  data_scaled = gdr.ProminentPoints(use=True, num_pts=25).fit_transform(data_scaled)
  return data_scaled

In [ ]:
def norm(data1, data2):

  return np.linalg.norm(np.array(data1)-np.array(data2)) 

In [ ]:
def get_mutual_features(pairs_dict):
  pairs_features_dict = {}
  for name, train_test in pairs_dict.items():

    print(name)
    pairs_features_dict.setdefault(name,{})

    for tt, df in train_test.items():
      print(tt)
      feature_dict = {}
      df_size = df.shape[0]
      #finding persistence column
      df_persistence_column = [col for col in df.columns if 'persistence' in col]

      for index, row in df.iterrows():
        
        if index % 10 == 0:
          print('Progress: {}/{}'.format(index,df_size))

        if 'DTM' in name:
          diag_X = [np.array(row[df_persistence_column[0]])[:-1]]
          diag_Y = [np.array(row[df_persistence_column[1]])[:-1]]
        else:
          diag_X = [np.array(row[df_persistence_column[0]])]
          diag_Y = [np.array(row[df_persistence_column[1]])]
      
        #preprocessing
        diag_X = data_preprocessing(diag_X)
        diag_Y = data_preprocessing(diag_Y)

        #computing norms
        #landscape
        ls_norm = norm(row['Landscape_x'], row['Landscape_y'])

        #silhouette norm 2
        sh_norm = norm(row['Silhouette_x'], row['Silhouette_y'])

        #entropy
        et_norm = norm(row['Entropy_vector_x'], row['Entropy_vector_y'])
        #Betti curves
        bc_norm = norm(row['Betti_curves_x'], row['Betti_curves_y'])

        #distances
        #bottleneck
        BD = gdr.BottleneckDistance(epsilon=.001)
        BD.fit(diag_X)
        bd = BD.transform(diag_Y)

        #wasserstein
        WD = gdr.WassersteinDistance(internal_p=2, order=2)
        WD.fit(diag_X)
        wd = WD.transform(diag_Y)

        #kernels
        ##PWG 
        PWG = gdr.PersistenceWeightedGaussianKernel(bandwidth=0.01, kernel_approx=None,\
                                            weight=lambda x: np.arctan(np.power(x[1], 1)))
        PWG.fit(diag_X)
        pwg = PWG.transform(diag_Y)

        ##PSS
        PSS = gdr.PersistenceScaleSpaceKernel(bandwidth=1.)
        PSS.fit(diag_X)
        pss = PSS.transform(diag_Y)

        ##PF
        PF = gdr.PersistenceFisherKernel(bandwidth_fisher=1., bandwidth=1.)
        PF.fit(diag_X)
        pf = PF.transform(diag_Y)

        ##SW
        SW = gdr.SlicedWassersteinKernel(bandwidth=.1, num_directions=100)
        SW.fit(diag_X)
        sw = SW.transform(diag_Y)

        #adding to dict
        feature_dict.setdefault(index, {})
        feature_dict[index]['Landscape_norm'] = ls_norm

        feature_dict[index]['Silhouette_norm'] = sh_norm

        feature_dict[index]['Entropy_norm1'] = et_norm

        feature_dict[index]['Betti_curves_norm'] = bc_norm

        feature_dict[index]['PWG_kernel'] = pwg[0][0]
        feature_dict[index]['PSS_kernel'] = pss[0][0]
        feature_dict[index]['PF_kernel'] = pf[0][0]
        feature_dict[index]['SW_kernel'] = sw[0][0]

        feature_dict[index]['Bottleneck_distance'] = bd[0][0]
        feature_dict[index]['Wasserstein_distance'] = wd[0][0]
      
      dict_df = pd.DataFrame.from_dict(feature_dict).transpose()
      joined_df = df.join(dict_df)
      pairs_features_dict[name][tt] = joined_df
  print('Done!')
  return pairs_features_dict

In [ ]:
features_pair_dict = get_mutual_features(pairs_df_dict)

af_orig
train
Progress: 0/140
Progress: 10/140
Progress: 20/140
Progress: 30/140
Progress: 40/140
Progress: 50/140
Progress: 60/140
Progress: 70/140
Progress: 80/140
Progress: 90/140
Progress: 100/140
Progress: 110/140
Progress: 120/140
Progress: 130/140
test
Progress: 0/38
Progress: 10/38
Progress: 20/38
Progress: 30/38
af_sparse
train
Progress: 0/140
Progress: 10/140
Progress: 20/140
Progress: 30/140
Progress: 40/140
Progress: 50/140
Progress: 60/140
Progress: 70/140
Progress: 80/140
Progress: 90/140
Progress: 100/140
Progress: 110/140
Progress: 120/140
Progress: 130/140
test
Progress: 0/38
Progress: 10/38
Progress: 20/38
Progress: 30/38
af_DTM
train
Progress: 0/140
Progress: 10/140
Progress: 20/140
Progress: 30/140
Progress: 40/140
Progress: 50/140
Progress: 60/140
Progress: 70/140
Progress: 80/140
Progress: 90/140
Progress: 100/140
Progress: 110/140
Progress: 120/140
Progress: 130/140
test
Progress: 0/38
Progress: 10/38
Progress: 20/38
Progress: 30/38
tf_orig
train
Progress: 0/140


In [ ]:
features_pair_dict['af_orig']['test'].head()

,name_x,data_persistence_x,Landscape_x,Silhouette_x,Entropy_vector_x,Betti_curves_x,Entropy_scalar_x,name_y,data_persistence_y,Landscape_y,Silhouette_y,Entropy_vector_y,Betti_curves_y,Entropy_scalar_y,ref_cover,Landscape_norm,Silhouette_norm,Entropy_norm1,Betti_curves_norm,PWG_kernel,PSS_kernel,PF_kernel,SW_kernel,Bottleneck_distance,Wasserstein_distance
0,All_Tomorrow_s_Parties,"[[0.1020223645, 0.10319306360000001], [0.10279...","[[0.0027412049, 0.0054824097, 0.0082236146, 0....","[[0.0, 5.2211000000000005e-05, 0.0001044219, 0...","[[0.18016593420000002, 0.18016593420000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0,...",[5.4899468328],All_Tomorrow_s_Parties,"[[0.0706149257, 0.0855859672], [0.0857599404, ...","[[0.0025188813000000003, 0.0050377626000000005...","[[0.0, 0.0001104923, 0.0002209846, 0.000331476...","[[0.2372037535, 0.2372037535, 0.2372037535, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[6.4666554449],1,5.028544,1.435878,10.562546,43.520110,29.631879,1.429135,0.999795,3.054103e-05,0.331601,0.542649
1,All_Tomorrow_s_Parties,"[[0.1020223645, 0.10319306360000001], [0.10279...","[[0.0027412049, 0.0054824097, 0.0082236146, 0....","[[0.0, 5.2211000000000005e-05, 0.0001044219, 0...","[[0.18016593420000002, 0.18016593420000002, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0,...",[5.4899468328],The_Boxer,"[[0.0843538408, 0.09618679200000001], [0.08540...","[[0.0025404340000000003, 0.0050808681, 0.00762...","[[0.0, 8.285040000000001e-05, 0.0001657009, 0....","[[0.36733056480000004, 0.36733056480000004, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[8.5317217936],0,2.766110,1.082271,37.888930,112.361025,3.132768,3.095388,0.999102,1.368433e-08,0.217217,0.572054
2,All_Along_The_Watchtower,"[[0.11884320570000001, 0.12036253520000001], [...","[[0.0027605234, 0.005521046700000001, 0.008281...","[[0.0, 6.603200000000001e-05, 0.0001422879, 0....","[[0.1404820797, 0.26634294950000004, 0.2663429...","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 4.0, 6.0,...",[4.7489072438],All_Along_The_Watchtower,"[[0.09851607200000001, 0.1022839812], [0.12726...","[[0.0025176595, 0.005035319, 0.0075529786, 0.0...","[[0.0, 8.93917e-05, 0.0001787835, 0.0002681752...","[[0.356727573, 0.356727573, 0.356727573, 0.356...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[8.3603092356],1,2.775650,1.112641,31.972310,93.642939,0.181489,1.631701,0.999548,5.382215e-07,0.218366,0.448463
3,All_Along_The_Watchtower,"[[0.11884320570000001, 0.12036253520000001], [...","[[0.0027605234, 0.005521046700000001, 0.008281...","[[0.0, 6.603200000000001e-05, 0.0001422879, 0....","[[0.1404820797, 0.26634294950000004, 0.2663429...","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 4.0, 6.0,...",[4.7489072438],All_Along_The_Watchtower,"[[0.10637688520000001, 0.1173920511], [0.13850...","[[0.0024423498, 0.0048846996, 0.0073270495, 0....","[[0.0, 5.2218e-05, 0.0001342909, 0.0002459114,...","[[0.3044932302, 0.3044932302, 0.6253215379, 0....","[[1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,...",[8.1758471256],1,3.490777,1.191165,27.725781,90.127687,0.411908,1.615032,0.999477,4.492986e-07,0.218366,0.471553
4,All_Along_The_Watchtower,"[[0.11884320570000001, 0.12036253520000001], [...","[[0.0027605234, 0.005521046700000001, 0.008281...","[[0.0, 6.603200000000001e-05, 0.0001422879, 0....","[[0.1404820797, 0.26634294950000004, 0.2663429...","[[1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 4.0, 6.0,...",[4.7489072438],All_Along_The_Watchtower,"[[0.0628478412, 0.0643986914], [0.0715882831, ...","[[0.001759179, 0.0035183579, 0.0052775369, 0.0...","[[0.0, 5.41092e-05, 0.00010821850000000001, 0....","[[0.3333524932, 0.3333524932, 0.3333524932, 0....","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0,...",[5.8114011233],1,3.882808,1.291520,16.448706,79.265377,6.410811,0.854012,0.999874,4.169831e-05,0.280352,0.443714


#Saving datasets

In [ ]:
import os

root_dir = '/content/drive/MyDrive/Colab Notebooks/CourseWork/final_pair_datasets_test015/'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

In [ ]:
for name, train_test in features_pair_dict.items():
  for tt, df in train_test.items():
    df.to_json(root_dir + name + '_' + tt + '.json')